In [1]:
import io
from PIL import Image
import os
import wandb
import glob
import torch
import monai
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

%matplotlib inline

In [2]:
torch.manual_seed(1024)
np.random.seed(1024)
device = torch.device(
    "cuda:0"
    if torch.cuda.is_available()
    else
    "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda:0 device


In [3]:
class SegDataset(Dataset):
    def __init__(self, data_root, transform, target_transform, train=True):
        self.data_root = data_root
        self.transform = transform
        self.target_transform = target_transform
        self.train = train
        self.gt_files_path = []
        # find all patient directories
        patient_directories = glob.glob(os.path.join(self.data_root, 'patient*'))
        # find all files with the suffix _gt.npy
        for patient_directory in patient_directories:
            per_patient_file_path = glob.glob(os.path.join(patient_directory, '*_gt.npy'))
            for path in per_patient_file_path:
                self.gt_files_path.append(path)
        
    def __len__(self):
        return len(self.gt_files_path)
    
    def __getitem__(self, index):
        gt_image_path = self.gt_files_path[index]
        image_path = gt_image_path[:-7] + ".npy"
        image = np.load(image_path)
        gt_image = np.load(gt_image_path)
        image = torch.tensor(image[None,:,:]).float()
        gt_image = torch.tensor(gt_image).long()
            
        # Convert the ground truth label to one-hot encoding
        one_hot_label = torch.nn.functional.one_hot(gt_image, num_classes=4)

        # Transpose the tensor to have dimensions (C, H, W)
        one_hot_label = one_hot_label.permute(2, 0, 1)

        # Remove the background channel (dimension 0)
        one_hot_label = one_hot_label[1:, :, :]
        
        # Use seed to make sure image and target has same transform
        seed = np.random.randint(2147483647)
        random.seed(seed)
        torch.manual_seed(seed)
        image = self.transform(image)
        random.seed(seed)
        torch.manual_seed(seed)
        target = self.target_transform(one_hot_label)
        
        return image, target

In [4]:
transform = transforms.Compose([
    transforms.Resize([224, 224])
])

dataset = SegDataset(data_root = './database/testing', 
                     transform = transform, 
                     target_transform = transform)

test_loader = DataLoader(dataset, batch_size=8, shuffle=False)


In [5]:
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels
    classes=3,                      # model output channels (number of classes)
)

model.load_state_dict(torch.load('./model/unet-test/model_best.pth'))
model.to(device)
model.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [6]:
run = wandb.init(
    project="Unet-Test",
    # Track hyperparameters and run metadata
    config={})

2023-05-30 11:26:05,145 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: ming686 (deeplearning-med). Use `wandb login --relogin` to force relogin


In [7]:
def vis_img(img, mask):
    # img: (B, 256, 64, 64), {: (B, 1, 256, 256)
    print(f"{img.shape=}, {mask.shape=}")
    img = np.squeeze(img)
    mask = np.squeeze(mask)
    plt.figure()
    plt.imshow(img, 'gray')
    overlay_mask_0 = np.ma.masked_where(mask[0] == 0, img)
    overlay_mask_1 = np.ma.masked_where(mask[1] == 0, img)
    overlay_mask_2 = np.ma.masked_where(mask[2] == 0, img)
    plt.imshow(overlay_mask_0, 'Greens', alpha = 0.7, clim=[0,1], interpolation='nearest')
    plt.imshow(overlay_mask_1, 'Reds', alpha = 0.7, clim=[0,1], interpolation='nearest')
    plt.imshow(overlay_mask_2, 'Purples', alpha = 0.7, clim=[0,1], interpolation='nearest')
    buffer = io.BytesIO()
    plt.savefig(buffer, format='jpeg')
    buffer.seek(0)

    # Convert the in-memory buffer to a NumPy array
    image_array = np.array(Image.open(buffer))
    return image_array

In [37]:
# Initialize evaluation metrics
sum_dice = 0.0
sum_mean_iou = 0.0
num_samples = 0

dice_metric = monai.metrics.DiceMetric(include_background=True,
                                       reduction="mean", 
                                       ignore_empty=False)
#hausdorff_distance_metric = monai.metrics.HausdorffDistanceMetric(include_background=True, reduction="mean")
mean_iou_metric = monai.metrics.MeanIoU(include_background=True, reduction="mean", ignore_empty=False)

with torch.no_grad():
    last_image_batch = None
    last_gt_mask_batch = None
    last_pr_mask_batch = None
    for step, (img, gt) in enumerate(tqdm(test_loader)):
        img = img.to(device)
        gt = gt.to(device)
        mask = model(img)
        
        
        # Vis
        shaped_image = img.detach().cpu().numpy()[0][0]
        shaped_gt = gt.detach().cpu().numpy()[0]
        shaped_pr = mask.detach().cpu().numpy()[0]
        
        threshold = 0.95  # Set your desired threshold value 
        binary_mask = (shaped_pr > threshold)
        
       # Compute the Dice coefficient for each sample in the batch
        prediction = (mask > threshold)
        dice = dice_metric([prediction], [gt])
        #hausdorff = hausdorff_distance_metric([prediction], [gt])
        mean_iou = mean_iou_metric([prediction], [gt])

        # Accumulate the Dice scores and update the number of samples
        print(dice)
        print(mean_iou)
        sum_dice += dice.sum().item()
        sum_mean_iou += mean_iou.sum().item()
        
        num_samples += 8
        
        # ground_truth = vis_img(shaped_image, shaped_gt)
        # predicted = vis_img(shaped_image, binary_mask)
        # # Log
        # wandb.log({"Result": [wandb.Image(ground_truth, caption="ground truth"), 
        #                       wandb.Image(predicted, caption="prediction")]})

# Compute the mean metrics
mean_dice = sum_dice / num_samples
mean_iou = sum_mean_iou / num_samples

print(f"Mean Dice coefficient: {mean_dice}")
print(f"Mean IoU: {mean_iou}")

  1%|▏         | 2/135 [00:00<00:16,  8.22it/s]

tensor([[0.8886, 0.7192, 1.0000, 0.9284, 1.0000, 0.8772, 0.8967, 0.9012]],
       device='cuda:0')
tensor([[0.7995, 0.5615, 1.0000, 0.8663, 1.0000, 0.7812, 0.8128, 0.8202]],
       device='cuda:0')
tensor([[0.9263, 0.8695, 0.9380, 0.9413, 0.0000, 0.8700, 0.0000, 0.8923]],
       device='cuda:0')
tensor([[0.8627, 0.7691, 0.8832, 0.8892, 0.0000, 0.7699, 0.0000, 0.8056]],
       device='cuda:0')


  3%|▎         | 4/135 [00:00<00:13,  9.61it/s]

tensor([[0.8672, 0.8845, 0.8771, 0.9270, 0.8914, 0.8222, 0.9372, 0.9017]],
       device='cuda:0')
tensor([[0.7656, 0.7928, 0.7811, 0.8639, 0.8041, 0.6981, 0.8818, 0.8210]],
       device='cuda:0')
tensor([[0.8066, 0.8666, 0.9179, 0.9133, 0.9154, 0.0000, 0.8796, 0.0000]],
       device='cuda:0')
tensor([[0.6759, 0.7646, 0.8483, 0.8404, 0.8440, 0.0000, 0.7851, 0.0000]],
       device='cuda:0')
tensor([[0.9362, 0.9448, 0.9163, 0.9365, 0.8741, 0.8918, 0.9491, 0.9120]],
       device='cuda:0')
tensor([[0.8800, 0.8954, 0.8456, 0.8805, 0.7764, 0.8047, 0.9031, 0.8383]],
       device='cuda:0')


  6%|▌         | 8/135 [00:00<00:11, 10.73it/s]

...

tensor([[0.0000, 0.9361, 0.9265, 0.9345, 0.6832, 0.9186, 0.9346, 0.9316]],
       device='cuda:0')
tensor([[0.0000, 0.8798, 0.8630, 0.8771, 0.5189, 0.8495, 0.8773, 0.8719]],
       device='cuda:0')
tensor([[0.6486, 0.9056, 0.6688, 0.9148, 0.9002, 0.8989, 0.0000, 0.9435]],
       device='cuda:0')
tensor([[0.4800, 0.8274, 0.5024, 0.8429, 0.8186, 0.8164, 0.0000, 0.8931]],
       device='cuda:0')
tensor([[0.9197, 0.5517, 0.9344, 0.8880, 0.9365, 0.9462, 0.9403, 0.9452]],
       device='cuda:0')
tensor([[0.8514, 0.3809, 0.8768, 0.7985, 0.8806, 0.8979, 0.8874, 0.8961]],
       device='cuda:0')


 96%|█████████▋| 130/135 [00:10<00:00, 12.21it/s]

tensor([[0.9561, 0.8972, 0.5503, 0.5975, 0.9241, 0.8333, 0.9463, 0.9163]],
       device='cuda:0')
tensor([[0.9159, 0.8136, 0.3796, 0.4260, 0.8589, 0.7143, 0.8981, 0.8455]],
       device='cuda:0')
tensor([[0.9177, 0.9519, 0.9208, 0.9249, 0.9238, 0.5574, 0.5259, 0.9434]],
       device='cuda:0')
tensor([[0.8479, 0.9082, 0.8532, 0.8603, 0.8583, 0.3864, 0.3567, 0.8929]],
       device='cuda:0')
tensor([[0.9254, 0.9118, 0.8955, 0.9211, 0.9228, 0.9520, 0.9437, 0.7335]],
       device='cuda:0')
tensor([[0.8611, 0.8379, 0.8108, 0.8537, 0.8567, 0.9084, 0.8935, 0.5791]],
       device='cuda:0')


100%|██████████| 135/135 [00:11<00:00, 11.95it/s]

tensor([[0.8602, 0.9156, 0.9155, 0.7961, 0.9133, 0.0000, 0.9015, 0.5534]],
       device='cuda:0')
tensor([[0.7546, 0.8444, 0.8442, 0.6613, 0.8405, 0.0000, 0.8206, 0.3825]],
       device='cuda:0')
tensor([[0.9176, 0.9299, 0.9081, 0.8735, 0.6994, 0.9025, 0.8437, 0.9013]],
       device='cuda:0')
tensor([[0.8477, 0.8689, 0.8316, 0.7755, 0.5378, 0.8223, 0.7296, 0.8203]],
       device='cuda:0')
tensor([[0.8533, 0.9107, 0.8587, 0.8610, 0.8899, 0.9043, 0.8011, 0.7864]],
       device='cuda:0')
tensor([[0.7442, 0.8361, 0.7525, 0.7559, 0.8016, 0.8253, 0.6682, 0.6480]],
       device='cuda:0')
tensor([[0.9396, 0.4747, 0.7869, 0.8928]], device='cuda:0')
tensor([[0.8861, 0.3112, 0.6486, 0.8063]], device='cuda:0')
Mean Dice coefficient: 0.7969261840537742
Mean IoU: 0.7174175191808629
